# `mammos_entity.io`: reading and writing entities

In [1]:
from pathlib import Path

import mammos_entity as me
import mammos_units as u
from mammos_entity.io import EntityCollection

## Supported file format

`mammos_entity.io` can read and write csv and yaml files containing entity like objects (entities, quantities, or other array-like data). For entities and quantities information about ontology and units are included as additional metadata. Details of the file formats are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html).

We create some artificial data that we can write to file:

In [2]:
Ms = me.Ms([600, 650, 700], "kA/m", description="Magnetization in kA/m")
T = me.T([1, 2, 3])
theta_angle = [0, 0.5, 0.7] * u.rad
demag_factor = me.Entity("DemagnetizingFactor", [1 / 3, 1 / 3, 1 / 3])
comments = ["Some comment", "Some other comment", "A third comment"]

## MaMMoS CSV

### Writing
We can write data to a csv file as shown in the following cell. Names of the keyword arguments determine column names in the file.

In [3]:
me.io.entities_to_file(
    "example.csv",
    "Example csv file.\nThe description can have multiple lines.\n\nLines can also be empty.",
    Ms=Ms,
    T=T,
    angle=theta_angle,
    demag_factor=demag_factor,
    comment=comments,
)

This has produced the following file:

In [4]:
print(Path("example.csv").read_text())

#mammos csv v2
#----------------------------------------
# Example csv file.
# The description can have multiple lines.
# 
# Lines can also be empty.
#----------------------------------------
#SpontaneousMagnetization,ThermodynamicTemperature,,DemagnetizingFactor,
#Magnetization in kA/m,,,,
#https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25,https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f,,https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e,
#kA / m,K,rad,,
Ms,T,angle,demag_factor,comment
600.0,1.0,0.0,0.3333333333333333,Some comment
650.0,2.0,0.5,0.3333333333333333,Some other comment
700.0,3.0,0.7,0.3333333333333333,A third comment



### Reading
We can read it back in and get a container object (called EntityCollection) containing all columns:

In [5]:
content = me.io.entities_from_file("example.csv")
content

EntityCollection(
    description='Example csv file.
 The description can have multiple lines.
 
 Lines can also be empty.
',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m', description='Magnetization in kA/m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object),
)

The recommended way of accessing the data is by using the individual elements. This preserves the correct data type:

In [6]:
content.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m', description='Magnetization in kA/m')

In [7]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K')

In [8]:
content.angle

<Quantity [0. , 0.5, 0.7] rad>

In [9]:
content.demag_factor

Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333]))

In [10]:
content.comment

array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object)

The `description` argument contains the description found in the metadata lines:

In [11]:
content.description

'Example csv file.\n The description can have multiple lines.\n \n Lines can also be empty.\n'

We can also get a `pandas` dataframe of the data we have read. This is designed as a convenience functions but due to limitation of `pandas` we loose ontology information. This is why we recommend using the individual elements directly where possible. The columns names consist of short name and units (where columns have a unit):

In [12]:
content.to_dataframe()

,Ms (kA / m),T (K),angle (rad),demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


We can also get a dataframe without units in the column names:

In [13]:
content.to_dataframe(include_units=False)

,Ms,T,angle,demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Reading with `pandas`
If we only need the numerical data but not the entity information, we can also read the csv file with pandas:

In [14]:
import pandas as pd

pd.read_csv("example.csv", comment="#")

,Ms,T,angle,demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Create, modify and save an `EntityCollection`

Manipulating an `EntityCollection` directly is not possible. Entities can however be overwritten or added to an existing collection.

In [15]:
Ms_kAm = content.Ms
Ms_Am = me.Ms(Ms_kAm, unit="A/m")
content.Ms = Ms_Am
content.A = me.A([6e-12, 7e-12, 8e-12])
content

EntityCollection(
    description='Example csv file.
 The description can have multiple lines.
 
 Lines can also be empty.
',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600000., 650000., 700000.]), unit='A / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([6.e-12, 7.e-12, 8.e-12]), unit='J / m'),
)

It is also possible to initialize an empty `EntityCollection` and to add entities as its attributes.

In [16]:
short_collection = EntityCollection()
short_collection.Ms = me.Ms(1.5e6)
short_collection.A = me.A(8e-12)
short_collection

EntityCollection(
    description='',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=np.float64(1500000.0), unit='A / m'),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=np.float64(8e-12), unit='J / m'),
)

To write an `EntityCollection` to disk use the following command.

In [17]:
me.io.entities_to_file(
    "modified_example.csv",
    "Modified version of `example.csv`.\nMs has been converted to A/m.",
    **content._elements_dictionary,
)
pd.read_csv("modified_example.csv", comment="#")

,Ms,T,angle,demag_factor,comment,A
0,600000.0,1.0,0.0,0.333333,Some comment,6.000000e-12
1,650000.0,2.0,0.5,0.333333,Some other comment,7.000000e-12
2,700000.0,3.0,0.7,0.333333,A third comment,8.000000e-12


## MaMMoS YAML

### Writing
We can write data to a yaml file (extension .yaml or .yml) as shown in the following cell. Names of the keyword arguments determine keys under `data:` in the file.

In [18]:
me.io.entities_to_file(
    "example.yaml",
    "Example yaml file.\nThe description can have multiple lines.\n\nLines can also be empty.",
    Ms=Ms,
    T=T,
    angle=theta_angle,
    demag_factor=demag_factor,
    comment=comments,
)

This has produced the following file:

In [19]:
print(Path("example.yaml").read_text())

metadata:
  version: v1
  description: |-
    Example yaml file.
    The description can have multiple lines.

    Lines can also be empty.
data:
  Ms:
    ontology_label: SpontaneousMagnetization
    description: Magnetization in kA/m
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: kA / m
    value: [600.0, 650.0, 700.0]
  T:
    ontology_label: ThermodynamicTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [1.0, 2.0, 3.0]
  angle:
    ontology_label: null
    description: null
    ontology_iri: null
    unit: rad
    value: [0.0, 0.5, 0.7]
  demag_factor:
    ontology_label: DemagnetizingFactor
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e
    unit: ''
    value: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
  comment:
    ontology_labe

### Reading
We can read it back in and get a container object (called EntityCollection) containing all columns:

In [20]:
content = me.io.entities_from_file("example.yaml")
content

EntityCollection(
    description='Example yaml file.
The description can have multiple lines.

Lines can also be empty.',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m', description='Magnetization in kA/m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=['Some comment', 'Some other comment', 'A third comment'],
)

We can access elements as show for the csv file before, e.g.

In [21]:
content.to_dataframe()

,Ms (kA / m),T (K),angle (rad),demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Entities of different shape and length

Unlike csv the yaml format supports entities with different lengths and shape.

In [22]:
me.io.entities_to_file("example2.yml", T=me.T([10, 20, 30]), Tc=me.Tc(300), Ms=me.Ms([[1, 2], [3, 4]]))

In [23]:
print(Path("example2.yml").read_text())

metadata:
  version: v1
  description: null
data:
  T:
    ontology_label: ThermodynamicTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [10.0, 20.0, 30.0]
  Tc:
    ontology_label: CurieTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_6b5af5a8_a2d8_4353_a1d6_54c9f778343d
    unit: K
    value: 300.0
  Ms:
    ontology_label: SpontaneousMagnetization
    description: ''
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: A / m
    value: [[1.0, 2.0], [3.0, 4.0]]



In [24]:
content = me.io.entities_from_file("example2.yml")
content

EntityCollection(
    description='',
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([10., 20., 30.]), unit='K'),
    Tc=Entity(ontology_label='CurieTemperature', value=np.float64(300.0), unit='K'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([[1., 2.],
       [3., 4.]]), unit='A / m'),
)

We can acces individual elements of the returned object as before.

In [25]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([10., 20., 30.]), unit='K')

In [26]:
content.Tc

Entity(ontology_label='CurieTemperature', value=np.float64(300.0), unit='K')

In [27]:
content.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([[1., 2.],
       [3., 4.]]), unit='A / m')

Conversion to a dataframe is however generally not possible as the shapes are incompatible with a `pandas.DataFrame`.

## Converting unformatted files and tables to `mammos csv`

Users may wish to update other files to the `mammos csv` format in order to make use of the additional functionality. 
Details of the file format are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html).

Converting your “raw” data into this format involves three main steps:

1. Load your file into python (e.g. with `pandas`).

2. Create an `Entity`, `quanity`, or similar out of each column (by assigning the correct ontology term and/or units).

3. Export the result with `entities_to_csv`.


First let's create a file so we can see an example of how to do the conversion. We will create the following structure:
```dat
1 10.0 1.6083568305976572 -16778187.088808443
1 9.0 1.6083393931987826 -15498304.121589921
...
```

This file is quite basic, in particular, there no headers, no units, no ontology information, and the use of the space as separator rather than a comma.

Only the user knows what each of the columns are.
In this example first column is the configuration type, the second column is the value of $\mu_0 H_{\mathsf{ext}}$ in Tesla, the third column is the magnetic polarisation in Tesla and the last column is the energy density in J/m$^3$.

In [28]:
Path("example.dat").write_text("""\
1 10.0 1.6083568305976572 -16778187.088808443
1 9.0 1.6083393931987826 -15498304.121589921
1 8.0 1.6083184361075116 -14218436.37373519
1 7.0 1.608292941666901 -12938587.029585946
1 6.0 1.6082614950059932 -11658760.230932372
""")

224

We can use `pandas` to read the file into python:

In [29]:
df = pd.read_csv("example.dat", sep=" ", names=["configuration_type", "mu0_Hext", "Js", "energy_density"])
df

,configuration_type,mu0_Hext,Js,energy_density
0,1,10.0,1.608357,-1.677819e+07
1,1,9.0,1.608339,-1.549830e+07
2,1,8.0,1.608318,-1.421844e+07
3,1,7.0,1.608293,-1.293859e+07
4,1,6.0,1.608261,-1.165876e+07


To rewrite this in the `mammos csv` format, we then need to associate each column with an entity, quantity, or another python object. Now is also time to do any data manipulation (such as changing units).

In this example we:
- Convert configuration type to a `numpy` array.
- Convert magnetic flux density ($\mu_0 H_{\mathsf{ext}}$) to the external magnetic field Entity using `mammos_units` for the relevant conversions.
- Convert magnetic polarisation to the corresponding entity.
- Convert energy density to the corresponding entity.

In [30]:
configuration_type = df["configuration_type"].to_numpy()
H = me.Entity(
    ontology_label="ExternalMagneticField",
    value=(df["mu0_Hext"].to_numpy() * u.T).to(u.A / u.m, equivalencies=u.magnetic_flux_field()),
    unit=u.A / u.m,
)
Js = me.Entity(
    ontology_label="MagneticPolarisation",
    value=df["Js"],
    unit=u.T,
)
energy_density = me.Entity(ontology_label="EnergyDensity", value=df["energy_density"], unit=u.J / u.m**3)

We can now write the `mammos csv`:

In [31]:
me.io.entities_to_file(
    "example.csv",
    "This file contains...",
    configuration_type=configuration_type,
    H=H,
    Js=Js,
    energy_density=energy_density,
)

Looking at the file produced we can see the data is now in the correct format with the ontology information included:

In [32]:
print(Path("example.csv").read_text())

#mammos csv v2
#----------------------------------------
# This file contains...
#----------------------------------------
#,ExternalMagneticField,MagneticPolarisation,EnergyDensity
#,,,
#,https://w3id.org/emmo/domain/magnetic_material#EMMO_da08f0d3-fe19-58bc-8fb6-ecc8992d5eb3,https://w3id.org/emmo#EMMO_74a096dd_cc83_4c7e_b704_0541620ff18d,https://w3id.org/emmo/domain/magnetic_material#EMMO_56258d3a-f2ee-554e-af99-499dd8620457
#,A / m,T,J / m3
configuration_type,H,Js,energy_density
1,7957747.150262763,1.6083568305976572,-16778187.088808443
1,7161972.435236487,1.6083393931987826,-15498304.12158992
1,6366197.72021021,1.6083184361075116,-14218436.37373519
1,5570423.005183934,1.608292941666901,-12938587.029585946
1,4774648.290157658,1.6082614950059932,-11658760.230932372



## Merging two `EntityCollection`

Reading a `mammos_entity` `csv` or `yaml` file will create an `EntityCollection` object as shown in the section above. It is possible to merge two of them using the `mammos_entity.merge` function.

In [29]:
me.io.entities_to_file(
    "A.csv",
    "This file contains computed coercivites",
    x_pos=me.Entity("Length", [-10.0, -10.0, -10.0, -10.0, -10.0], "mm"),
    y_pos=me.Entity("Length", [-10.0, -15.0, -20.0, -25.0, -30.0], "mm"),
    a=me.Entity("LocalLatticeConstantA", [8.7838, 8.7835, 8.7834, 8.7828, 8.7824], "Angstrom"),
    c=me.Entity("LocalLatticeConstantC", [12.1634, 12.1629, 12.1626, 12.1614, 12.1611], "Angstrom"),
    volume=me.Entity("CellVolume", [938.4688596, 938.3661812, 938.3216703, 938.1009148, 938.056412], "Angstrom^3"),
    Ms=me.Ms([1205044.626, 1203469.633, 1202605.731, 1198792.8, 1197854.062]),
    A=me.A([7.071954208, 7.053474008, 7.043351497, 6.998792402, 6.987851124], "pJ/m"),
    K1=me.Ku([3.411298313, 3.388623141, 3.37627427, 3.3225042, 3.309445639], "MJ/m3"),
    Ha=me.Entity(
        "AnisotropyField",
        [4505434.713046, 4481343.84430899, 4468220.34398305, 4411045.5691224, 4397151.94905576],
        "A/m",
    ),
)

In [30]:
me.io.entities_to_file(
    "B.csv",
    "Integral of Absolute Difference",
    x_pos=me.Entity("Length", [-1, -1, -1, -1, -1], "cm"),
    y_pos=me.Entity("Length", [-10.0, -15.0, -20.0, -25.0, -30.0], "mm"),
    integral_abs_diff=[1.176182592576099, 1.174635419947949, 1.1535128973539177, 1.204475631146945, 1.1041682311364889],
)

In [31]:
A_collection = me.io.entities_from_file("A.csv")
A_collection

EntityCollection(
    x_pos=Entity(ontology_label='Length', value=array([-10., -10., -10., -10., -10.]), unit='mm'),
    y_pos=Entity(ontology_label='Length', value=array([-10., -15., -20., -25., -30.]), unit='mm'),
    a=Entity(ontology_label='LocalLatticeConstantA', value=array([8.7838, 8.7835, 8.7834, 8.7828, 8.7824]), unit='Angstrom'),
    c=Entity(ontology_label='LocalLatticeConstantC', value=array([12.1634, 12.1629, 12.1626, 12.1614, 12.1611]), unit='Angstrom'),
    volume=Entity(ontology_label='CellVolume', value=array([938.4688596, 938.3661812, 938.3216703, 938.1009148, 938.056412 ]), unit='Angstrom3'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1205044.626, 1203469.633, 1202605.731, 1198792.8  , 1197854.062]), unit='A / m'),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([7.07195421, 7.05347401, 7.0433515 , 6.9987924 , 6.98785112]), unit='pJ / m'),
    K1=Entity(ontology_label='UniaxialAnisotropyConstant', value=array([3.4112983

In [32]:
B_collection = me.io.entities_from_file("B.csv")
B_collection

EntityCollection(
    x_pos=Entity(ontology_label='Length', value=array([-1., -1., -1., -1., -1.]), unit='cm'),
    y_pos=Entity(ontology_label='Length', value=array([-10., -15., -20., -25., -30.]), unit='mm'),
    integral_abs_diff=array([1.17618259, 1.17463542, 1.1535129 , 1.20447563, 1.10416823]),
)

In [33]:
merged_collection = me.merge(A_collection, B_collection, how="inner")
merged_collection

EntityCollection(
    x_pos=Entity(ontology_label='Length', value=array([-10., -10., -10., -10., -10.]), unit='mm'),
    y_pos=Entity(ontology_label='Length', value=array([-10., -15., -20., -25., -30.]), unit='mm'),
    a=Entity(ontology_label='LocalLatticeConstantA', value=array([8.7838, 8.7835, 8.7834, 8.7828, 8.7824]), unit='Angstrom'),
    c=Entity(ontology_label='LocalLatticeConstantC', value=array([12.1634, 12.1629, 12.1626, 12.1614, 12.1611]), unit='Angstrom'),
    volume=Entity(ontology_label='CellVolume', value=array([938.4688596, 938.3661812, 938.3216703, 938.1009148, 938.056412 ]), unit='Angstrom3'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1205044.626, 1203469.633, 1202605.731, 1198792.8  , 1197854.062]), unit='A / m'),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([7.07195421, 7.05347401, 7.0433515 , 6.9987924 , 6.98785112]), unit='pJ / m'),
    K1=Entity(ontology_label='UniaxialAnisotropyConstant', value=array([3.4112983

In [34]:
merged_collection.to_dataframe()

,x_pos (mm),y_pos (mm),a (Angstrom),c (Angstrom),volume (Angstrom3),Ms (A / m),A (pJ / m),K1 (MJ / m3),Ha (A / m),integral_abs_diff
0,-10.0,-10.0,8.7838,12.1634,938.468860,1205044.626,7.071954,3.411298,4.505435e+06,1.176183
1,-10.0,-15.0,8.7835,12.1629,938.366181,1203469.633,7.053474,3.388623,4.481344e+06,1.174635
2,-10.0,-20.0,8.7834,12.1626,938.321670,1202605.731,7.043351,3.376274,4.468220e+06,1.153513
3,-10.0,-25.0,8.7828,12.1614,938.100915,1198792.800,6.998792,3.322504,4.411046e+06,1.204476
4,-10.0,-30.0,8.7824,12.1611,938.056412,1197854.062,6.987851,3.309446,4.397152e+06,1.104168


It is important to note that:
- The `merge` function is a thin wrapper around the `pandas.merge` function and the merge behavior depends on the additional keyword arguments passed to `merge` (which are forwarded to `pandas.merge` internally) on top of two `EntityCollection` to be merged. Please refer to the [panda.merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) documentation.
- If both collections contain entities with the same ontology label but different units, the `right` entity is automatically converted to the unit of the `left` entity before merging.